# Соколов Александр, лабораторная 3, вариант 22

In [94]:
import numpy as np
from scipy.linalg import lu_factor
from itertools import product

Условия задачи 3.1:

In [95]:
#Размерность матрицы.
n = 5 
#Число компонент вектора свободных членов.
N = 22 

#Погрешность:
tol = 1e-5

#Коэффициенты:
c = lambda i,j: 0.1*N*i*j

#Расчет матрицы А и вектора своб. членов:
A = np.array([[150/(13*c(i, j)**3 + 777*c(i, j)) for i in range(1,n+1)] for j in range(1,n+1)])
b = np.array([float(N) for i in range(n)])

In [96]:
print('A:', A, '\n', 'b:', b)

A: [[8.11765623e-02 3.31404412e-02 1.69192370e-02 9.55612683e-03
  5.80270793e-03]
 [3.31404412e-02 9.55612683e-03 3.73543330e-03 1.77413409e-03
  9.64518577e-04]
 [1.69192370e-02 3.73543330e-03 1.28981557e-03 5.77568461e-04
  3.04369529e-04]
 [9.55612683e-03 1.77413409e-03 5.77568461e-04 2.52382871e-04
  1.31396836e-04]
 [5.80270793e-03 9.64518577e-04 3.04369529e-04 1.31396836e-04
  6.80083968e-05]] 
 b: [22. 22. 22. 22. 22.]


In [97]:
x = np.linalg.solve(A, b)

In [98]:
print('Solution:', x)

Solution: [ 6.12459681e+04 -2.97214518e+06  3.14972246e+07 -9.81621158e+07
  8.59407521e+07]


In [99]:
cond = np.linalg.cond(A, p=np.inf)

In [100]:
print('Condition number of matrix A:', cond)

Condition number of matrix A: 8422543.146718727


In [101]:
bs = np.repeat(b, n).reshape((n, n))
diag = np.diag([eps]*5)
bs = bs + diag

In [102]:
eps = 0.001
sols = []
for b in bs:
    sols.append(np.linalg.solve(A, b))

In [103]:
norms = list(map(lambda x_i: max(abs(x - x_i))/max(abs(x)), sols))

In [104]:
norms = np.array(norms, dtype=np.float64)

In [105]:
norms

array([1.11440990e-07, 7.11246434e-06, 8.17713815e-05, 2.62811226e-04,
       2.33499312e-04])

In [106]:
comp = np.argsort(norms)[-1]

In [107]:
assert max(np.abs(norms))==norms[comp]

In [108]:
print('most influential component of vector b:', comp)

most influential component of vector b: 3


In [109]:
estimate = cond*(max(np.abs(b - bs[comp]))/max(abs(b)))

In [110]:
print('theoretical estimate:', estimate, '\n', 'practical estimate:', norms[comp])

theoretical estimate: 382.825469148176 
 practical estimate: 0.00026281122569218966


Практическая оценка много меньше теоретической. Теоретическая оценка получилась большой из-за большого числа обусловленности (порядка миллиона), которое участвует в формуле для теоретической оценки. Но оценка на практике не превосходит теоретического супремума, поэтому решение, полученное для зашумленных данных (которое соответствует теоретической оценке), является адекватным.

# 3.4

In [111]:
print('A:', A)

A: [[8.11765623e-02 3.31404412e-02 1.69192370e-02 9.55612683e-03
  5.80270793e-03]
 [3.31404412e-02 9.55612683e-03 3.73543330e-03 1.77413409e-03
  9.64518577e-04]
 [1.69192370e-02 3.73543330e-03 1.28981557e-03 5.77568461e-04
  3.04369529e-04]
 [9.55612683e-03 1.77413409e-03 5.77568461e-04 2.52382871e-04
  1.31396836e-04]
 [5.80270793e-03 9.64518577e-04 3.04369529e-04 1.31396836e-04
  6.80083968e-05]]


In [112]:
def getUpper(matrix, n):
    for i, j in product(range(n), range(n)):
        matrix[i, j] = 0 if i>j else matrix[i, j]
    return matrix
def getLower(matrix, n):
    for i, j in product(range(n), range(n)):
        if i<j:
            matrix[i, j]=0
        elif i==j:
            matrix[i, j]=1
        else:
            matrix[i, j] = matrix[i, j]
    return matrix

In [113]:
LU, p = lu_factor(A)

In [114]:
lower = getLower(LU.copy(), n)

In [115]:
upper = getUpper(LU.copy(), n)

In [116]:
b_gauss = np.linalg.inv(lower)@b

In [117]:
sol_lu = np.linalg.solve(upper, b_gauss)

In [118]:
print('solved with lu-decomposition:', sol_lu, '\n', 'solved with np.solve:', x)

solved with lu-decomposition: [ 6.12354116e+04 -2.97144700e+06  3.14891978e+07 -9.81363177e+07
  8.59178314e+07] 
 solved with np.solve: [ 6.12459681e+04 -2.97214518e+06  3.14972246e+07 -9.81621158e+07
  8.59407521e+07]


In [119]:
sols = []
sols.append(b_gauss[-1]/upper[-1][-1])
for i in range(n-2, -1, -1):
    sols.append((b_gauss[i] - np.dot(upper[i][i+1:], sols[-1::-1]))/upper[i][i])

In [120]:
print('manual back pass:', sols[-1::-1])

manual back pass: [61235.41155157041, -2971447.0020514177, 31489197.791344415, -98136317.69788097, 85917831.35456848]
